<a href="https://colab.research.google.com/github/nicovakr/Pytorch-init/blob/master/CNN_PyTorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torchvision
import os
import tarfile

from torchvision.datasets.utils import download_url
from torch.utils.data import random_split

In [2]:
project_name = '05-cifar10-cnn'

In [3]:
# DL the dataset
dataset_url = "https://s3.amazonaws.com/fast-ai-imageclas/cifar10.tgz"
download_url(dataset_url, '.')

  0%|          | 0/135107811 [00:00<?, ?it/s]

In [4]:
# extract from archive
with tarfile.open('./cifar10.tgz', 'r:gz') as tar:
  tar.extractall(path='./data')

In [5]:
data_dir = './data/cifar10'

print(os.listdir(data_dir))
classes = os.listdir(data_dir + "/train")
print(classes)

['test', 'train']
['bird', 'horse', 'truck', 'cat', 'deer', 'ship', 'frog', 'dog', 'airplane', 'automobile']


In [6]:
airplane_files = os.listdir(data_dir + "/train/airplane")
print('Number of training example for airplanes:', len(airplane_files))
print(airplane_files[:5])

Number of training example for airplanes: 5000
['0093.png', '4349.png', '2774.png', '2128.png', '0552.png']
